## Live Notebook

This notebook is intended for live-coding during the JWebbinar. You should only do this if you feel comfortable trying to keep up! If you'd rather have an already-populated notebook, go to [specutils.ipynb](specutils.ipynb) instead, which you can execute as we go.

#  Data Analysis Tools JWebbinar: Specutils

![Specutils: An Astropy Package for Spectroscopy](specutils_logo.png)


This notebook provides an overview of the Astropy coordinated package `specutils`.  While this notebook is intended as an interactive introduction to specutils at the time of its writing, the canonical source of information for the package is the latest version's documentation: 

https://specutils.readthedocs.io

`specutils` should already be in your JWebbinar environment.  If you wish to install locally, you can follow the instructions in [the installation section of the specutils docs](https://specutils.readthedocs.io/en/latest/installation.html).

Once specutils is installed, fundamental imports necessary for this notebook are possible:

In [ ]:
import numpy as np

import astropy.units as u
from astropy.io import fits

import specutils
from specutils import Spectrum1D, SpectralRegion, analysis, manipulation, fitting
specutils.__version__

In [ ]:
# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt


# for showing quantity units on axes automatically:
from astropy.visualization import quantity_support
quantity_support();

# Background/Spectroscopic ecosystem

The large-scale plan for spectroscopy support in the Astropy project is outlined in  the [Astropy Proposal For Enhancement 13](https://github.com/astropy/astropy-APEs/blob/main/APE13.rst).  In summary, this APE13 lays out three broad packages:

* `specutils` - a Python package containing the basic data structures for representing spectroscopic data sets, as well as a suite of fundamental spectroscopic analysis tools to work with these data structures.
* `specreduce` - a general Python package to reduce raw astronomical spectral images to 1d spectra (represented as `specutils` objects).
* `specviz` - a Python package (or possibly suite of packages) for visualization of astronomical spectra. While Astropy has not developed this, STScI has built [`jdaviz`](https://jdaviz.readthedocs.io/en/latest/) for this purpose (especially, but not exclusively, for JWST).


The first is the subject of this notebook.

# Fundamentals of specutils

## Objects for representing spectra

The most fundamental purpose of `specutils` is to contain the shared Python-level data structures for storing astronomical spectra.  It is important to recognize that this is not the same as the *on-disk* representation.  As desecribed later specutils provides loaders and writers for various on-disk representations, with the intent that they all load to a common set of in-memory/Python interfaces.  Those intefaces (implemented as Python classes) are described in detail in the [relevant section of the documentation](https://specutils.readthedocs.io/en/latest/types_of_spectra.html), which contains this diagram:

![Specutils Classes](specutils_classes_diagrams.png)

The core principal is that all of these representations contain a `spectral_axis` attribute as well as a `flux` attribute (as well as optional matching `uncertainty`).  The former is often wavelength for OIR spectra, but might be frequency or energy for e.g. Radio or X-ray spectra.  Regardless of which spectral axis is used, the class attempts to interpret it appropriately, using the features of `astropy.Quantity` to distinguish different types of axes.  Similarly, `flux` may or may not be a traditional astronomical `flux` unit (e.g. Jy or  erg sec$^{-1}$ cm$^{-2}$ angstrom$^{-1}$), but is treated as the portion of the spectrum that acts in that manner.  The various classes are then distinguished by whether these attributes are one-dimensional or not, and how to map the `spectral_axis` dimensionality onto the `flux`.  The simplest case (and the one primarily considered here) is the scalar `Spectrum1D` case, which is a single spectrum with a matched-size `flux` and `spectral_axis`.

## Basics of creating Spectrum1D Objects

If your spectrum is in a format that specutils understands, loading it is very straightforward.  There are times when you want a bit more control, though, so lets look at loading from a file and creating an object directly from arrays:

## Loading spectra from files

Specutils comes with readers for a variety of spectral data formats (including loaders for future JWST instruments). While support for specific formats depends primarily from users (like you!) providing readers, you may find that one has already been implemented for your favorite spectrum format.  As an example, we consider a simulated high-redshift (z > 1) galaxy like that you might see from NIRSpec:

In [ ]:
url_to_download = 'https://stsci.box.com/shared/static/b22b1fzhimtdqfp8597m4bg67kovvauu.fits'

To see the full list of formats readable in your current version of specutils, see the table at the bottom of the `Spectrum1d.read` method:

# Creating a Spectrum "by-hand"

If you have a format that is not compatible, or you want to do some sort of customization of the loading process, spectra can be created directly from aastropy quantities (which are basically arrays with associated units). Here we'll show how you can do that, using data from the same file above: 

### Exercise

If you have your own spectroscopic data, try loading a file here using either one of the built-in loaders, or the `Spectrum1D` interface, and plotting it.  If you don't have your own data on-hand, you can try downloading something of interest via a public archive (e.g., public HST data using MAST), or you can try with an [SDSS galaxy spectrum](https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12) included in this repo (look for the file `example_sdss_spectrum.fits`).  Once you've got your dataset, try loading it into a `Spectrum1D` object whichever of the ways above makes most sense to you.

If you have time and enough knowledge of the format, try loading your spectrum *both* ways - with a built-in loader and manually creating a `Spectrum1D`.

## Working with Units and Spectral Axes 

We created `Spectrum1D` just as Quantity arrays, so they can be treated just as `Quantity` objects when convenient with unit conversions and the like:

There's even fast-accessors to make some of this more convenient:

But under the hood this are are fully-featured WCS following the [Astropy APE14](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst) WCS interface along with the [GWCS](https://gwcs.readthedocs.io/) package. So you can use that to do conversions to and from spectral to pixel axes:

The other dimension is the flux - that requires slightly more complex transformation because it matters where in the spectrum you are to do the flux transformation:

But specutils makes this much easier!

# Arithmetic on Spectra

Specutils provides a lot of functionality for manipulating spectra.  In general these follow the pattern of creating *new* specutils objects with the results of the operation instead of in-place operations.

The most straightforward of operations are arithmetic manipulations.  In general these follow patterns that are based on fundametal arithmetic. E.g.:

However, when there is ambiguity in your intent - for example, two spectra with different units where it is not clear what the desired output is - errors are generally produced instead of the code attempting to guess:

Resolving this requires explicit conversion:

### Exercises

1. Try continuum *normalizing* using the above assumed continuum level, instead of subtracting. 
2. You can tell by-eye that the continuum isn't a flat level, but has a slope to it.  Try subtracting a *sloping* continuum instead of a single value (it's fine to just estimate it by-eye, fancier continuum subtracting will come later).

## SNR and Uncertainties

Now lets try a simple calculation: the S/N of this spectrum. While pipeline-output JWST files will have uncertainties, for this example we are using a basic simulation without the uncertainities. Hence we start using an SNR estimate that follows a [straightforward algorithm detailed in the literature](https://www.stecf.org/software/ASTROsoft/DER_SNR/).

That's straightforward enough, but does not use any uncertainty information beyond the spectrum itself. To mock up what real JWST might look like (or how you would provide your *own* uncertainties), we can create a new Spectrum1D object with by-hand added uncertainties:

After that, it is a one-liner to compute the S/N directly, which will use the uncertainty already in the spectrum: